In [151]:
# Declarations
import glob
import json
import os
import numpy as np
import pandas as pd
import re

from os.path import basename

In [152]:
path_ref = '../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL'
path_ref_name = 'Thumb-Arm-Atmega-Leon3-RiscV-RECIPE_DECIMAL-'

In [153]:
import time
import pandas as pd
from IPython.core.magic import register_cell_magic
from IPython.display import display, HTML

# CSV file to save execution times
csv_filename = path_ref_name + "06-00-ExecutionTime.csv"

# Initialize the CSV file with header if it doesn't exist
try:
    df = pd.read_csv(csv_filename)
except FileNotFoundError:
    df = pd.DataFrame(columns=["cell_number", "execution_time"])
    df.to_csv(csv_filename, index=False)

# Function to log execution times
def log_execution_time(cell_number, execution_time):
    df = pd.read_csv(csv_filename)
    new_row = pd.DataFrame({"cell_number": [cell_number], "execution_time": [execution_time]})
    df = pd.concat([df, new_row], ignore_index=True)
    df.to_csv(csv_filename, index=False)

# Decorator for the magic function
@register_cell_magic
def log_time(line, cell):
    start_time = time.time()
    exec(cell, globals())
    end_time = time.time()
    execution_time = end_time - start_time

    # Current cell number
    cell_number = int(line) if line.isdigit() else None
    if cell_number is None:
        display(HTML("<b>Warning:</b> Cell number not provided!"))
        return

    # Log the execution time
    log_execution_time(cell_number, execution_time)
    display(HTML(f"<b>Cell {cell_number}:</b> Execution time = {execution_time:.6f} seconds"))

# To use the magic function in the notebook
# %%log_time 1
# cell code

In [154]:
%%log_time 1

nameFileV = []
pathFileV = []
rootFileV = []
nameFileM = []
pathFileM = []
rootFileM = []
nameFileS = []
pathFileS = []
rootFileS = []
nameFileSV = []
pathFileSV = []
rootFileSV = []

# path = os.getcwd()
for root, dirs, files in os.walk(path_ref):
    for file in files:
        if file.endswith("ScalarSize.csv"):
            nameFileS.append(file)
            pathFileS.append(os.path.join(root, file))
            rootFileS.append(root)
        elif file.endswith("ScalarVariable.csv"):
            nameFileSV.append(file)
            pathFileSV.append(os.path.join(root, file))
            rootFileSV.append(root)
        elif file.endswith("VectorNew.csv"):
            nameFileV.append(file)
            pathFileV.append(os.path.join(root, file))
            rootFileV.append(root)
        elif file.endswith("MatrixNew.csv"):
            nameFileM.append(file)
            pathFileM.append(os.path.join(root, file))
            rootFileM.append(root)

/tmp/ipykernel_3737/4257067356.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)


In [155]:
len(pathFileS)

79

In [156]:
len(pathFileSV)

34

In [157]:
len(pathFileV)

50

In [158]:
len(pathFileM)

29

In [159]:
'''
for i in range(0, len(pathFileS)):
    if os.path.exists(pathFileS[i]):
        os.remove(pathFileS[i])
        
for i in range(0, len(pathFileSV)):
    if os.path.exists(pathFileSV[i]):
        os.remove(pathFileSV[i])
        
for i in range(0, len(pathFileV)):
    if os.path.exists(pathFileV[i]):
        os.remove(pathFileV[i])
        
for i in range(0, len(pathFileM)):
    if os.path.exists(pathFileM[i]):
        os.remove(pathFileM[i])
'''

'\nfor i in range(0, len(pathFileS)):\n    if os.path.exists(pathFileS[i]):\n        os.remove(pathFileS[i])\n        \nfor i in range(0, len(pathFileSV)):\n    if os.path.exists(pathFileSV[i]):\n        os.remove(pathFileSV[i])\n        \nfor i in range(0, len(pathFileV)):\n    if os.path.exists(pathFileV[i]):\n        os.remove(pathFileV[i])\n        \nfor i in range(0, len(pathFileM)):\n    if os.path.exists(pathFileM[i]):\n        os.remove(pathFileM[i])\n'

In [160]:
%%log_time 2

# ScalarSize file
for i in range(0, len(pathFileS)):
    path = pathFileS[i]
    try:
        dfSTemp = pd.read_csv(path, sep=',',engine='python')
    except:
        print(path)

    # dfSTemp = dfSTemp.rename(columns={dfSTemp.columns[0]:'ID_VAL'})    
    
    for j in range(1, len(dfSTemp.columns)):
        dfSTemp = dfSTemp.rename(columns={dfSTemp.columns[j]:'SS_VAL_'+str(j)})
    
    if os.name == 'nt':
        spR = rootFileS[i].split('\\')
    else:
        spR = rootFileS[i].split('/')
        
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    # board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # Bambu (HW)
    board = spR[-4] # SW
    function = spR[-5]
    dfSTemp['DEVICE'] = device
    dfSTemp['BOARD'] = board
    dfSTemp['FUNCTION'] = function
    dfSTemp['DATA_TYPE'] = datatype
    # dfSTemp['OPT_FLAG'] = optFlag
    # dfSTemp['MV_SIZE'] = matrixVectorSize
    dfSTemp['inputName'] = dfSTemp['inputName'].astype(str)
    dfSTemp['inputName'] = dfSTemp['inputName'].map(lambda x: x.lstrip('values_'))
    dfSTemp['inputName'] = dfSTemp['inputName'].astype(int)
    dfSTemp.sort_values('inputName')
    
    if i == 0:
        dfSTot = dfSTemp
    else:
        frames = [dfSTot, dfSTemp]
        dfSTot = pd.concat(frames)
        
dfSTot = dfSTot.fillna(0)

In [161]:
dfSTot

,inputName,SS_VAL_1,DEVICE,BOARD,FUNCTION,DATA_TYPE
0,11,17,Leon3,Leon3,matrix_mult_atmega_fault,double
1,19,3,Leon3,Leon3,matrix_mult_atmega_fault,double
2,12,5,Leon3,Leon3,matrix_mult_atmega_fault,double
3,38,10,Leon3,Leon3,matrix_mult_atmega_fault,double
4,81,24,Leon3,Leon3,matrix_mult_atmega_fault,double
...,...,...,...,...,...,...
95,3,3,RiscV,RiscV,shell_sort,float
96,98,14,RiscV,RiscV,shell_sort,float
97,29,18,RiscV,RiscV,shell_sort,float
98,80,6,RiscV,RiscV,shell_sort,float


In [162]:
%%log_time 3

# ScalarVariable file
for i in range(0, len(pathFileSV)):
    path = pathFileSV[i]
    try:
        dfSVTemp = pd.read_csv(path, sep=',',engine='python')
    except:
        print(path)
    
    # dfSVTemp = dfSVTemp.rename(columns={dfSVTemp.columns[0]:'ID_VAL'})
    
    for j in range(1, len(dfSVTemp.columns)):
        dfSVTemp = dfSVTemp.rename(columns={dfSVTemp.columns[j]:'SV_VAL_'+str(j)})
        
    if os.name == 'nt':
        spR = rootFileSV[i].split('\\')
    else:
        spR = rootFileSV[i].split('/')
    
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    # board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # Bambu (HW)
    board = spR[-4] # SW
    function = spR[-5]
    dfSVTemp['DEVICE'] = device
    dfSVTemp['BOARD'] = board
    dfSVTemp['FUNCTION'] = function
    dfSVTemp['DATA_TYPE'] = datatype
    # dfSVTemp['OPT_FLAG'] = optFlag
    # dfSVTemp['MV_SIZE'] = matrixVectorSize
    dfSVTemp['inputName'] = dfSVTemp['inputName'].astype(str)
    dfSVTemp['inputName'] = dfSVTemp['inputName'].map(lambda x: x.lstrip('values_'))
    dfSVTemp['inputName'] = dfSVTemp['inputName'].astype(int)
    dfSVTemp.sort_values('inputName')
    
    if i == 0:
        dfSVTot = dfSVTemp
    else:
        frames = [dfSVTot, dfSVTemp]
        dfSVTot = pd.concat(frames)

dfSVTot = dfSVTot.fillna(0)

In [163]:
dfSVTot

,inputName,SV_VAL_1,DEVICE,BOARD,FUNCTION,DATA_TYPE
0,11,22,Leon3,Leon3,lud,double
1,19,27,Leon3,Leon3,lud,double
2,12,10,Leon3,Leon3,lud,double
3,38,9,Leon3,Leon3,lud,double
4,81,15,Leon3,Leon3,lud,double
...,...,...,...,...,...,...
95,3,24,RiscV,RiscV,bs,float
96,98,3,RiscV,RiscV,bs,float
97,29,3,RiscV,RiscV,bs,float
98,80,13,RiscV,RiscV,bs,float


In [164]:
%%log_time 4

# Vector file
for i in range(0, len(pathFileV)):
    path = pathFileV[i]
    dfVTemp = pd.read_csv(path, sep=',',engine='python')
    
    dfVTemp = dfVTemp.rename(columns={dfVTemp.columns[0]:'inputName'})
    
    for j in range(1, len(dfVTemp.columns)):
        dfVTemp = dfVTemp.rename(columns={dfVTemp.columns[j]:'V_VAL_'+str(j)})
    
    if os.name == 'nt':
        spR = rootFileV[i].split('\\')
    else:
        spR = rootFileV[i].split('/')
    
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    # board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # Bambu (HW)
    board = spR[-4] # SW
    function = spR[-5]
    dfVTemp['DEVICE'] = device
    dfVTemp['BOARD'] = board
    dfVTemp['FUNCTION'] = function
    dfVTemp['DATA_TYPE'] = datatype
    # dfVTemp['OPT_FLAG'] = optFlag
    # dfVTemp['MV_SIZE'] = matrixVectorSize
    # dfVTemp['ID_VAL'] = dfVTemp['ID_VAL'].astype(str)
    dfVTemp['inputName'] = dfVTemp['inputName'].map(lambda x: x.lstrip('values_') if isinstance(x, str) else x)
    dfVTemp['inputName'] = dfVTemp['inputName'].astype(int)
    dfVTemp.sort_values('inputName')
    
    if i == 0:
        dfVTot = dfVTemp
    else:
        frames = [dfVTot, dfVTemp]
        dfVTot = pd.concat(frames)

dfVTot = dfVTot.fillna(0)

In [165]:
dfVTot

,inputName,V_VAL_1,V_VAL_2,V_VAL_3,V_VAL_4,V_VAL_5,V_VAL_6,V_VAL_7,V_VAL_8,V_VAL_9,...,V_VAL_27,V_VAL_28,V_VAL_29,V_VAL_30,V_VAL_31,V_VAL_32,DEVICE,BOARD,FUNCTION,DATA_TYPE
0,19,-5.434138e+18,9.761585e+16,4.079293e+17,-5.699568e+18,5.639897e+18,9.033023e+18,8.192734e+18,3.370946e+18,-8.288799e+18,...,0.0,0.0,0.0,0.0,0.0,0.0,Leon3,Leon3,bsort100,double
1,12,3.549122e+18,-6.588195e+18,-3.869135e+17,-6.069483e+18,6.883611e+18,-5.966515e+18,5.942673e+18,-8.742032e+18,-3.031813e+18,...,0.0,0.0,0.0,0.0,0.0,0.0,Leon3,Leon3,bsort100,double
2,38,-2.995901e+18,-1.640477e+18,-2.823985e+18,-4.824965e+18,6.285613e+18,4.794959e+18,1.997976e+18,-1.690951e+18,6.688827e+18,...,0.0,0.0,0.0,0.0,0.0,0.0,Leon3,Leon3,bsort100,double
3,81,7.392561e+18,5.859770e+18,-3.112809e+18,8.427490e+17,-2.658247e+18,5.072116e+18,7.813931e+18,-2.893187e+18,-5.437471e+18,...,0.0,0.0,0.0,0.0,0.0,0.0,Leon3,Leon3,bsort100,double
4,5,1.092416e+18,-2.479435e+18,-7.322973e+18,4.325103e+18,-9.141665e+17,-2.127761e+18,8.041895e+18,2.998952e+17,-4.573151e+18,...,0.0,0.0,0.0,0.0,0.0,0.0,Leon3,Leon3,bsort100,double
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,3,-7.167662e+08,1.132358e+09,8.192887e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,RiscV,RiscV,shell_sort,float
95,98,1.348037e+08,-6.640992e+08,-1.493482e+09,1.769519e+09,1.414456e+09,-1.048165e+09,1.773689e+09,-1.681504e+09,-2.120018e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,RiscV,RiscV,shell_sort,float
96,29,-1.345173e+09,-3.576364e+08,-9.515675e+08,1.899560e+09,7.272661e+08,1.194980e+09,-1.013699e+09,-1.641731e+09,1.399037e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,RiscV,RiscV,shell_sort,float
97,80,9.313834e+08,-1.533059e+09,7.966424e+07,2.834458e+08,9.040505e+08,9.578276e+08,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,RiscV,RiscV,shell_sort,float


In [166]:
%%log_time 5

# Matrix file
pathFileM.pop(0)
rootFileM.pop(0)

for i in range(0, len(pathFileM)):
    path = pathFileM[i]
    dfMTemp = pd.read_csv(path, sep=',',engine='python')
    
    dfMTemp = dfMTemp.rename(columns={dfMTemp.columns[0]:'inputName'})
    
    for j in range(1, len(dfMTemp.columns)):
        dfMTemp = dfMTemp.rename(columns={dfMTemp.columns[j]:'M_VAL_'+str(j)})
    
    if os.name == 'nt':
        spR = rootFileM[i].split('\\')
    else:
        spR = rootFileM[i].split('/')
    
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    # board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # Bambu (HW)
    board = spR[-4] # SW
    function = spR[-5]
    dfMTemp['DEVICE'] = device
    dfMTemp['BOARD'] = board
    dfMTemp['FUNCTION'] = function
    dfMTemp['DATA_TYPE'] = datatype
    # dfMTemp['OPT_FLAG'] = optFlag
    # dfMTemp['MV_SIZE'] = matrixVectorSize
    # dfMTemp['ID_VAL'] = dfVTemp['ID_VAL'].astype(str)
    dfMTemp['inputName'] = dfMTemp['inputName'].map(lambda x: x.lstrip('values_') if isinstance(x, str) else x)
    dfMTemp['inputName'] = dfMTemp['inputName'].astype(int)
    dfMTemp.sort_values('inputName')
    
    if i == 0:
        dfMTot = dfMTemp
    else:
        frames = [dfMTot, dfMTemp]
        dfMTot = pd.concat(frames)

dfMTot = dfMTot.fillna(0)

In [167]:
dfMTot

,inputName,M_VAL_1,M_VAL_2,M_VAL_3,M_VAL_4,M_VAL_5,M_VAL_6,M_VAL_7,M_VAL_8,M_VAL_9,...,M_VAL_67,M_VAL_68,M_VAL_69,M_VAL_70,M_VAL_71,M_VAL_72,DEVICE,BOARD,FUNCTION,DATA_TYPE
0,19,0.0,0.0,197096224.0,0.000000e+00,486860448.0,41432920.0,0.0,210577568.0,0.0,...,4572013.0,0.0,0.0,0.0,6481190.5,54423660.0,Leon3,Leon3,matrix_mult_atmega_fault,float
1,12,165752384.0,0.0,0.0,2.134084e+08,438333184.0,0.0,19924766.0,376990976.0,634457472.0,...,302941024.0,0.0,167807296.0,121990512.0,0.0,646224512.0,Leon3,Leon3,matrix_mult_atmega_fault,float
2,38,0.0,0.0,0.0,0.000000e+00,13165755.0,0.0,4603877.0,6809795.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Leon3,Leon3,matrix_mult_atmega_fault,float
3,81,0.0,3921982.5,64912752.0,2.719199e+07,65995600.0,11256252.0,41353388.0,128462408.0,3418550.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Leon3,Leon3,matrix_mult_atmega_fault,float
4,5,0.0,0.0,554103104.0,0.000000e+00,592523136.0,0.0,0.0,801788672.0,147124848.0,...,337768800.0,76064936.0,283734464.0,0.0,0.0,939062336.0,Leon3,Leon3,matrix_mult_atmega_fault,float
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,3,417372032.0,0.0,255312640.0,8.444909e+07,587990144.0,30068468.0,105682056.0,538195904.0,0.0,...,353134240.0,0.0,340958240.0,81440696.0,0.0,345492736.0,RiscV,RiscV,matrix_mult,float
95,98,0.0,0.0,342072448.0,0.000000e+00,748836160.0,0.0,0.0,655996352.0,0.0,...,340191104.0,333074112.0,563424384.0,90022048.0,0.0,472455936.0,RiscV,RiscV,matrix_mult,float
96,29,0.0,22376028.0,39891816.0,9.567438e+05,108639560.0,0.0,37929472.0,77381824.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,RiscV,RiscV,matrix_mult,float
97,80,0.0,0.0,357655680.0,0.000000e+00,520602304.0,0.0,0.0,570910784.0,138368496.0,...,68441736.0,11414587.0,72644616.0,32190188.0,4522094.0,176778256.0,RiscV,RiscV,matrix_mult,float


In [168]:
dfSTot = dfSTot.reset_index(drop=True)
dfSVTot = dfSVTot.reset_index(drop=True)
dfVTot = dfVTot.reset_index(drop=True)
dfMTot = dfMTot.reset_index(drop=True)

In [169]:
dfSTot

,inputName,SS_VAL_1,DEVICE,BOARD,FUNCTION,DATA_TYPE
0,11,17,Leon3,Leon3,matrix_mult_atmega_fault,double
1,19,3,Leon3,Leon3,matrix_mult_atmega_fault,double
2,12,5,Leon3,Leon3,matrix_mult_atmega_fault,double
3,38,10,Leon3,Leon3,matrix_mult_atmega_fault,double
4,81,24,Leon3,Leon3,matrix_mult_atmega_fault,double
...,...,...,...,...,...,...
7895,3,3,RiscV,RiscV,shell_sort,float
7896,98,14,RiscV,RiscV,shell_sort,float
7897,29,18,RiscV,RiscV,shell_sort,float
7898,80,6,RiscV,RiscV,shell_sort,float


In [170]:
%%log_time 6

# To get the common columns between the DataFrames, we can use set intersection
common_columns = set(dfSTot.columns).intersection(set(dfSVTot.columns)).intersection(set(dfVTot.columns)).intersection(set(dfMTot.columns))

# Convert the intersection to a list
common_columns = list(common_columns)

# Merge the DataFrames using the common columns
dfTot = pd.merge(dfSTot, dfSVTot, on=common_columns, how='left', sort=False)
dfTot = pd.merge(dfTot, dfVTot, on=common_columns, how='left', sort=False) 
dfTot = pd.merge(dfTot, dfMTot, on=common_columns, how='left', sort=False) 

# Display the resulting DataFrame
# print(dfTot)

In [171]:
# dfSTot.to_csv("dfSTot.csv", sep=';')
# dfSVTot.to_csv("dfSVTot.csv", sep=';')
# dfVTot.to_csv("dfVTot.csv", sep=';')
# dfMTot.to_csv("dfMTot.csv", sep=';')

In [172]:
dfTot

,inputName,SS_VAL_1,DEVICE,BOARD,FUNCTION,DATA_TYPE,SV_VAL_1,V_VAL_1,V_VAL_2,V_VAL_3,...,M_VAL_63,M_VAL_64,M_VAL_65,M_VAL_66,M_VAL_67,M_VAL_68,M_VAL_69,M_VAL_70,M_VAL_71,M_VAL_72
0,11,17,Leon3,Leon3,matrix_mult_atmega_fault,double,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19,3,Leon3,Leon3,matrix_mult_atmega_fault,double,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12,5,Leon3,Leon3,matrix_mult_atmega_fault,double,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38,10,Leon3,Leon3,matrix_mult_atmega_fault,double,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,81,24,Leon3,Leon3,matrix_mult_atmega_fault,double,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7895,3,3,RiscV,RiscV,shell_sort,float,NaN,-7.167662e+08,1.132358e+09,8.192887e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7896,98,14,RiscV,RiscV,shell_sort,float,NaN,1.348037e+08,-6.640992e+08,-1.493482e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7897,29,18,RiscV,RiscV,shell_sort,float,NaN,-1.345173e+09,-3.576364e+08,-9.515675e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7898,80,6,RiscV,RiscV,shell_sort,float,NaN,9.313834e+08,-1.533059e+09,7.966424e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
%%log_time 7

dfTot = dfTot[ ['inputName','DEVICE','BOARD','FUNCTION','DATA_TYPE'] + [ col for col in dfTot.columns if (col != 'inputName' and col != 'DEVICE' and col != 'BOARD' and col != 'FUNCTION' and col != 'DATA_TYPE') ] ]

#### Realingn column

In [174]:
dfTot

,inputName,DEVICE,BOARD,FUNCTION,DATA_TYPE,SS_VAL_1,SV_VAL_1,V_VAL_1,V_VAL_2,V_VAL_3,...,M_VAL_63,M_VAL_64,M_VAL_65,M_VAL_66,M_VAL_67,M_VAL_68,M_VAL_69,M_VAL_70,M_VAL_71,M_VAL_72
0,11,Leon3,Leon3,matrix_mult_atmega_fault,double,17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19,Leon3,Leon3,matrix_mult_atmega_fault,double,3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12,Leon3,Leon3,matrix_mult_atmega_fault,double,5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38,Leon3,Leon3,matrix_mult_atmega_fault,double,10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,81,Leon3,Leon3,matrix_mult_atmega_fault,double,24,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7895,3,RiscV,RiscV,shell_sort,float,3,NaN,-7.167662e+08,1.132358e+09,8.192887e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7896,98,RiscV,RiscV,shell_sort,float,14,NaN,1.348037e+08,-6.640992e+08,-1.493482e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7897,29,RiscV,RiscV,shell_sort,float,18,NaN,-1.345173e+09,-3.576364e+08,-9.515675e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7898,80,RiscV,RiscV,shell_sort,float,6,NaN,9.313834e+08,-1.533059e+09,7.966424e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
%%log_time 8

dfTot.to_csv("TotalInput.csv", sep=';', index=False, encoding='utf-8')

In [ ]:
############ SPLIT INPUT ################

In [ ]:
# dfMTot.replace('matrix_mult_atmega_fault', 'matrix_mult', inplace=True)

In [176]:
import pandas as pd
import os

def split_csv_by_column(input_file, column_name):
    # Read the CSV file
    df = pd.read_csv(input_file, sep=';')

    # Get unique values in the specified column
    unique_values = df[column_name].unique()

    # Get the directory of the input file
    input_dir = os.path.dirname(input_file)

    # For each unique value, create a subset of the DataFrame and save it as a CSV file
    for value in unique_values:
        subset_df = df[df[column_name] == value]
        output_file = os.path.join(input_dir, f"{value}-RECIPE-DECIMAL-Input.csv")
        subset_df.to_csv(output_file, index=False)
        print(f"Created file: {output_file}")

input_file = 'Input/Thumb-Arm-RiscV-Leon3-Atmega-RECIPE-DECIMAL-Input.csv'  # replace with the path to your CSV file
column_name = 'DEVICE'  # replace with the name of the column to split by

split_csv_by_column(input_file, column_name)

Created file: Input/Leon3-RECIPE-DECIMAL-Input.csv
Created file: Input/Arm-RECIPE-DECIMAL-Input.csv
Created file: Input/Atmega328p-RECIPE-DECIMAL-Input.csv
Created file: Input/Thumb-RECIPE-DECIMAL-Input.csv
Created file: Input/RiscV-RECIPE-DECIMAL-Input.csv


In [21]:
####################### OLD CODE #############################

In [102]:
import pandas as pd
import numpy as np

'''
# Add Zeros
def merge_dataframes_fillna(df1, df2, merge_on):
    # Perform the merge with suffixes to handle duplicate columns
    merged_df = pd.merge(df1, df2, how='outer', on=merge_on, suffixes=('_df1', '_df2'))

    # Identify columns with suffixes
    suffixes = ('_df1', '_df2')
    common_columns = [col.replace(suffixes[0], '') for col in merged_df.columns if col.endswith(suffixes[0])]

    # Combine duplicate columns by filling missing values with zero
    for col in common_columns:
        col_df1 = f"{col}{suffixes[0]}"
        col_df2 = f"{col}{suffixes[1]}"
        
        # Convert columns to numeric type, if possible
        merged_df[col_df1] = pd.to_numeric(merged_df[col_df1], errors='coerce').fillna(0)
        merged_df[col_df2] = pd.to_numeric(merged_df[col_df2], errors='coerce').fillna(0)
        
        # Sum the values of the converted columns
        merged_df[col] = merged_df[col_df1] + merged_df[col_df2]
    
    # Remove the columns with suffixes
    columns_to_drop = [f"{col}{suffixes[0]}" for col in common_columns] + [f"{col}{suffixes[1]}" for col in common_columns]
    merged_df.drop(columns=columns_to_drop, inplace=True)

    # Fill missing values in other columns with zero
    merged_df.fillna(0, inplace=True)
    
    return merged_df
'''

def make_columns_unique(df):
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique():
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols
    return df

# Add NaN
def merge_dataframes_fillna(df1, df2, merge_on):
    # Make sure column names are unique in both DataFrames
    df1 = make_columns_unique(df1)
    df2 = make_columns_unique(df2)
    
    # Perform the merge with suffixes to handle duplicate columns
    merged_df = pd.merge(df1, df2, how='outer', on=merge_on, suffixes=('_df1', '_df2'))

    # Identify columns with suffixes
    suffixes = ('_df1', '_df2')
    common_columns = [col.replace(suffixes[0], '') for col in merged_df.columns if col.endswith(suffixes[0])]

    # Combine duplicate columns by filling missing values with NaN
    for col in common_columns:
        col_df1 = f"{col}{suffixes[0]}"
        col_df2 = f"{col}{suffixes[1]}"
        
        # Combine the columns, filling NaN where data is missing
        merged_df[col] = merged_df[col_df1].combine_first(merged_df[col_df2])

    # Remove the columns with suffixes
    columns_to_drop = [f"{col}{suffixes[0]}" for col in common_columns] + [f"{col}{suffixes[1]}" for col in common_columns]
    merged_df.drop(columns=columns_to_drop, inplace=True)

    return merged_df

In [103]:
merge_on = ['ID_VAL','DEVICE','BOARD','FUNCTION','DATA_TYPE','OPT_FLAG','MV_SIZE']

In [104]:
dfTot1 = merge_dataframes_fillna(dfSTot, dfSVTot, merge_on)

In [105]:
dfTot1

,ID_VAL,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,SS_VAL_1,SV_VAL_1
0,11,Leon3,Leon3,banker_algorithm,int16_t,optO2-09,32x32,17.0,NaN
1,12,Leon3,Leon3,banker_algorithm,int16_t,optO2-09,32x32,10.0,NaN
2,19,Leon3,Leon3,banker_algorithm,int16_t,optO2-09,32x32,18.0,NaN
3,38,Leon3,Leon3,banker_algorithm,int16_t,optO2-09,32x32,2.0,NaN
4,73,Arm,Arm,astar,int8_t,optO1-00,32x32,NaN,7.0
5,73,Arm,Arm,astar,int8_t,optO1-01,32x32,NaN,7.0
6,73,Arm,Arm,astar,int8_t,optO1-02,32x32,NaN,7.0
7,80,Arm,Arm,astar,int8_t,optO1-00,32x32,NaN,214.0
8,80,Arm,Arm,astar,int8_t,optO1-01,32x32,NaN,214.0
9,81,Leon3,Leon3,banker_algorithm,int16_t,optO2-09,32x32,22.0,NaN


In [106]:
dfTot1.columns

Index(['ID_VAL', 'DEVICE', 'BOARD', 'FUNCTION', 'DATA_TYPE', 'OPT_FLAG',
       'MV_SIZE', 'SS_VAL_1', 'SV_VAL_1'],
      dtype='object')

In [107]:
dfTot2 = merge_dataframes_fillna(dfTot1, dfVTot, merge_on)


In [108]:
dfTot2.columns

Index(['ID_VAL', 'DEVICE', 'BOARD', 'FUNCTION', 'DATA_TYPE', 'OPT_FLAG',
       'MV_SIZE', 'SS_VAL_1', 'SV_VAL_1', 'ID_VAL_1', 'V_VAL_1', 'V_VAL_2',
       'V_VAL_3', 'V_VAL_4', 'V_VAL_5', 'V_VAL_6', 'V_VAL_7', 'V_VAL_8',
       'V_VAL_9', 'V_VAL_10', 'V_VAL_11', 'V_VAL_12', 'V_VAL_13', 'V_VAL_14',
       'V_VAL_15', 'V_VAL_16', 'V_VAL_17', 'V_VAL_18', 'V_VAL_19', 'V_VAL_20',
       'V_VAL_21', 'V_VAL_22', 'V_VAL_23', 'V_VAL_24', 'V_VAL_25', 'V_VAL_26',
       'V_VAL_27', 'V_VAL_28', 'V_VAL_29', 'V_VAL_30', 'V_VAL_31', 'V_VAL_32'],
      dtype='object')

In [109]:
dfTot3 = dfTot2

In [110]:
dfTot3.columns

Index(['ID_VAL', 'DEVICE', 'BOARD', 'FUNCTION', 'DATA_TYPE', 'OPT_FLAG',
       'MV_SIZE', 'SS_VAL_1', 'SV_VAL_1', 'ID_VAL_1', 'V_VAL_1', 'V_VAL_2',
       'V_VAL_3', 'V_VAL_4', 'V_VAL_5', 'V_VAL_6', 'V_VAL_7', 'V_VAL_8',
       'V_VAL_9', 'V_VAL_10', 'V_VAL_11', 'V_VAL_12', 'V_VAL_13', 'V_VAL_14',
       'V_VAL_15', 'V_VAL_16', 'V_VAL_17', 'V_VAL_18', 'V_VAL_19', 'V_VAL_20',
       'V_VAL_21', 'V_VAL_22', 'V_VAL_23', 'V_VAL_24', 'V_VAL_25', 'V_VAL_26',
       'V_VAL_27', 'V_VAL_28', 'V_VAL_29', 'V_VAL_30', 'V_VAL_31', 'V_VAL_32'],
      dtype='object')

In [111]:
#convert year variable in df2 to integer
dfMTot['ID_VAL']=dfMTot['ID_VAL'].astype(str)
dfTot3['ID_VAL']=dfTot3['ID_VAL'].astype(str)

In [112]:
dfTot4 = merge_dataframes_fillna(dfTot3, dfMTot, merge_on)


In [113]:
dfTot4

,ID_VAL,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,SS_VAL_1,SV_VAL_1,V_VAL_1,...,M_VAL_64,M_VAL_65,M_VAL_66,M_VAL_67,M_VAL_68,M_VAL_69,M_VAL_70,M_VAL_71,M_VAL_72,ID_VAL_1
0,0,Arm,Arm,astar,int8_t,optO1-00,32x32,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0
1,0,Arm,Arm,astar,int8_t,optO1-01,32x32,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0
2,0,Arm,Arm,astar,int8_t,optO1-02,32x32,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0
3,0,Leon3,Leon3,banker_algorithm,int16_t,optO1-00,32x32,NaN,NaN,1725.0,...,9123.916016,4056.985596,3445.193359,3842.214111,0.000000,10366.075195,7450.783691,0.0,5630.526855,0.0
4,0,Leon3,Leon3,banker_algorithm,int16_t,optO1-01,32x32,NaN,NaN,1725.0,...,9123.916016,4056.985596,3445.193359,3842.214111,0.000000,10366.075195,7450.783691,0.0,5630.526855,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,99,Leon3,Leon3,banker_algorithm,int16_t,optO2-00,32x32,NaN,NaN,891.0,...,31102.449219,0.000000,3546.772949,0.000000,4676.758789,20752.488281,9838.529297,0.0,23316.640625,99.0
1185,99,Leon3,Leon3,banker_algorithm,int16_t,optO2-01,32x32,NaN,NaN,891.0,...,31102.449219,0.000000,3546.772949,0.000000,4676.758789,20752.488281,9838.529297,0.0,23316.640625,99.0
1186,99,Leon3,Leon3,banker_algorithm,int16_t,optO2-02,32x32,NaN,NaN,891.0,...,31102.449219,0.000000,3546.772949,0.000000,4676.758789,20752.488281,9838.529297,0.0,23316.640625,99.0
1187,99,Leon3,Leon3,banker_algorithm,int16_t,optO2-09,32x32,NaN,NaN,891.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0


In [114]:
dfTot4.columns

Index(['ID_VAL', 'DEVICE', 'BOARD', 'FUNCTION', 'DATA_TYPE', 'OPT_FLAG',
       'MV_SIZE', 'SS_VAL_1', 'SV_VAL_1', 'V_VAL_1',
       ...
       'M_VAL_64', 'M_VAL_65', 'M_VAL_66', 'M_VAL_67', 'M_VAL_68', 'M_VAL_69',
       'M_VAL_70', 'M_VAL_71', 'M_VAL_72', 'ID_VAL_1'],
      dtype='object', length=114)

In [115]:
dfTot5 = dfTot4.fillna(0)

In [117]:
dfTot5.to_csv("TotalInput.csv", sep=';', index=False, encoding='utf-8')

In [45]:
####################### Useful Code ##########################################

In [46]:
# Matrix divided for Boards

In [47]:
df1 = dfTot5.loc[(dfTot5['BOARD'] == '5SGXEA7N2F45C1-Stratix-V')]
df1.to_csv("TotalParameterMatrixBambuValue_Stratix.csv", sep=';')

In [48]:
df2 = dfTot5.loc[(dfTot5['BOARD'] == 'xc7a100t-1csg324-Artix-7')]
df2.to_csv("TotalParameterMatrixBambuValue_Artix.csv", sep=';')

In [49]:
df3 = dfTot5.loc[(dfTot5['BOARD'] == 'xc7vx690t-3ffg1930-Virtex-7')]
df3.to_csv("TotalParameterMatrixBambuValue_Virtex.csv", sep=';')

In [50]:
df4 = dfTot5.loc[(dfTot5['BOARD'] == 'xc7z020-1clg484-Zynq-7000')]
df4.to_csv("TotalParameterMatrixBambuValue_Zynq.csv", sep=';')

In [54]:
dfBamb = dfTot5.loc[(dfTot5['DEVICE'] == 'Bambu') & (dfTot5['BOARD'] == 'xc7a100t-1csg324-Artix-7')]

In [55]:
dfBamb.groupby(['DEVICE','BOARD','FUNCTION']).count()

DATA_TYPE  ID_VAL  DSP  BRAM  \
DEVICE BOARD                    FUNCTION                                      
Bambu  xc7a100t-1csg324-Artix-7 astar                100     100  100   100   
                                bellmanford          100     100  100   100   
                                bfsdfs               100     100  100   100   
                                binarysearch         100     100  100   100   
                                bubblesort           100     100  100   100   
                                dijkstra             100     100  100   100   
                                fft                  100     100  100   100   
                                fibcall              100     100  100   100   
                                floydwarshall        100     100  100   100   
                                gcd                  100     100  100   100   
                                insertionsort        100     100  100   100   
                                kruskal              100     100  100   100   
                                mergesort            100     100  100   100   
                                park_miller          100     100  100   100   
                                prime                100     100  100   100   
                                qsort                100     100  100   100   
                                quicksort            100     100  100   100   
                                selectionsort        100     100  100   100   
                                shell_sort           100     100  100   100   
                                sqrt                 100     100  100   100   
                                straight_sort        100     100  100   100   

                                               Flip_Flop  Area  min_Slack  \
DEVICE BOARD                    FUNCTION                                    
Bambu  xc7a100t-1csg324-Artix-7 astar                100   100        100   
                                bellmanford          100   100        100   
                                bfsdfs               100   100        100   
                                binarysearch         100   100        100   
                                bubblesort           100   100        100   
                                dijkstra             100   100        100   
                                fft                  100   100        100   
                                fibcall              100   100        100   
                                floydwarshall        100   100        100   
                                gcd                  100   100        100   
                                insertionsort        100   100        100   
                                kruskal              100   100        100   
                                mergesort            100   100        100   
                                park_miller          100   100        100   
                                prime                100   100        100   
                                qsort                100   100        100   
                                quicksort            100   100        100   
                                selectionsort        100   100        100   
                                shell_sort           100   100        100   
                                sqrt                 100   100        100   
                                straight_sort        100   100        100   

                                               Max_Freq  Time  Energy  ...  \
DEVICE BOARD                    FUNCTION                               ...   
Bambu  xc7a100t-1csg324-Artix-7 astar               100   100     100  ...   
                                bellmanford         100   100     100  ...   
                                bfsdfs              100   100     100  ...   
                                binarysearch        100   100     1

In [56]:
dfBamb.loc[dfTot5['FUNCTION'] == 'gcd'].groupby(['DEVICE','BOARD','FUNCTION','DATA_TYPE']).count()

,,,,ID_VAL,DSP,BRAM,Flip_Flop,Area,min_Slack,Max_Freq,Time,Energy,BugsDelivered,...,M_VAL_23,M_VAL_24,M_VAL_25,M_VAL_26,M_VAL_27,M_VAL_28,M_VAL_29,M_VAL_30,M_VAL_31,M_VAL_32
DEVICE,BOARD,FUNCTION,DATA_TYPE,,,,,,,,,,,,,,,,,,,,,
Bambu,xc7a100t-1csg324-Artix-7,gcd,int8_t,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
